# Deep Models: LSTM Forecasting Prototype

This notebook demonstrates a minimal workflow to train a small LSTM model for NAV forecasting using per-scheme features. It uses the `raptor.src.deep_models` helper which defers heavy dependencies (TensorFlow) until required.

Sections:
1. Setup & imports
2. Load sample features for one scheme
3. Create supervised sequences (window/horizon)
4. Train a small LSTM (if TensorFlow available)
5. Evaluate and visualize predictions


In [ ]:
# 1) Setup & imports
import os
from pathlib import Path
import logging

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from raptor.src import preprocessing, deep_models

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger('notebook.deep')

ROOT = Path('..').resolve()
FEATURES_DIR = ROOT / 'data' / 'features'
print('Features dir:', FEATURES_DIR)


ModuleNotFoundError: No module named 'raptor'

## 2) Load sample scheme features
Find one features_*.parquet file and load it. We will use its NAV series to construct sequences.


In [6]:
files = sorted(FEATURES_DIR.glob('features_*.parquet'))
assert files, 'No feature files found; run preprocessing.generate_all_features()'
sample_file = files[0]
df = pd.read_parquet(sample_file)
df = df.sort_values('date')

nav = df.set_index('date')['nav']
plt.plot(nav.iloc[-200:])
plt.title('Recent NAV (sample)')
plt.show()


NameError: name 'FEATURES_DIR' is not defined

## 3) Create supervised sequences (window/horizon)


In [7]:
WINDOW = 30
HORIZON = 7
X, y = deep_models.create_sequences(nav, window=WINDOW, horizon=HORIZON)
print('X shape, y shape:', X.shape, y.shape)

# use last 20% as validation split
split = int(len(X)*0.8)
X_train, X_val = X[:split], X[split:]
y_train, y_val = y[:split], y[split:]
print('Train/Val sizes:', len(X_train), len(X_val))


NameError: name 'deep_models' is not defined

## 4) Train small LSTM (if TensorFlow available)

The training will be skipped if TensorFlow is not installed in the environment. For a full experiment, install `tensorflow` and re-run.


In [8]:
try:
    model = deep_models.build_and_train_lstm(X_train, y_train, epochs=5, batch_size=32, verbose=1)
    trained = True
except Exception as e:
    print('Skipping LSTM training (missing TF or runtime issue):', e)
    model = None
    trained = False

print('Trained:', trained)


Skipping LSTM training (missing TF or runtime issue): name 'deep_models' is not defined
Trained: False


## 5) Evaluate and visualize predictions


In [ ]:
if trained and model is not None:
    preds = model.predict(X_val)
    # compare first horizon only for simplicity
    plt.figure(figsize=(10,4))
    plt.plot(y_val[:50,0], label='actual')
    plt.plot(preds[:50,0], label='pred')
    plt.legend()
    plt.title('LSTM: actual vs predicted (first horizon)')
    plt.show()
else:
    print('No trained model to evaluate; consider installing TensorFlow and re-running.')
